# Equivalent Algal Populations model of Phytoplankton Inherent Optical Properties: Two layered (coated) sphere code

At the core of the two layered code is a collection of numerical routines written in Fortran 77. These need to be wrapped into a python wrapper in order for the code to be able to recognise them on your machine. There are a number of online resources for instructions on how to do this. Here we share the process if you use the f2py module. Before you start though, make sure you have a Fortran compiler installed, we use gfortran.

In [5]:
from numpy import f2py

sourcefile = open('/.../Dmmex_R14B_4.f','rb') # change the filepath and name as appropriate
sourcecode = sourcefile.read()
f2py.compile(sourcecode, modulename='Dmmex_R14B_4') # you can name the wrapped module whatever you want

FileNotFoundError: [Errno 2] No such file or directory: '/.../Dmmex_R14B_4.f'

### Import the required modules to run the two layered code, including the wrapped fortran module:

In [6]:
import Dmmex_R14B_4
import scipy.io as io
import numpy as np

In [7]:
l = np.arange(.4, .905, .005) 
# l is the wavelength range and resolution. 
# Changing the above changes your 'int_val' value when normalising kshell, so:

int_val = 55 # refers to l[55] which is 675 nm. 255 for 1 nm resolution


In [15]:
Vs = 0.2 # this is the proportional volume of the shell sphere (Vs) to the core shell (Vc). Together they add up to 1.
Vc=1 - Vs
FR=(1- Vs) ** (1/ 3) # this is then the relative volume of core to shell

ci = 2e6 # 2.5 kg per m3 is 2.5e6 milligrams per m3. 
D_eff = np.arange(1.5, 2.5, 1) # These are the effective diameters that the code will produce IOPs for.
V_eff= 0.6 # distribution effective variance

mf = io.loadmat('/Users/lisl/Desktop/STUDENTS/FRIEDA/501nm_extended_e1701000.mat') # change to your RI filename

nmedia = 1.334 # the refractive index of water
wavelength = l/ nmedia

wvno = 2 * np.pi / wavelength # this is a Mie parameter for the scattering calculations 


### Hilbert transform functions are not standard with numpy, so declare it here:

In [16]:
# definition of the hilbert transform function

def analytic_signal(x):
    from scipy.fftpack import fft, ifft
    N = len(x)
    X = fft(x, N)
    h = np.zeros(N)
    h[0] = 1
    h[1:N//2] = 2* np.ones(N// 2-1)
    h[N// 2] = 1
    Z = X * h
    z = ifft(Z, N)
    return z

### Calculate all core and shell refractive indices: 

The imaginary refractive indices are interpolated onto the desired wavelengths, scaled as appropriate and then transformed to produce the corresponding spectral real refractive index for core and shell separately, also scaled as appropriate.

In [17]:
from scipy.interpolate import griddata

kcore = griddata(mf['RIs'][:, 5], mf['RIs'][:, 0], l, 'linear')
# the core imaginary refractive index (representing absorption) is small and has a predictable spectral shape
kshell_base = griddata(mf['RIs'][:, 5], mf['RIs'][:, 2], l, 'linear')
# the spectral shape of the shell imaginary refractive index is retrieved here, scaling it is next:

# this scales the imaginary RI to this theoretical maximum absorption of unpackaged chl a at 675 nm.
# Here Johnsen 1982 is used (0.027)
kshell_norm = (6.75e-7/ nmedia) * (0.027 * ci/ Vs) / (4 * np.pi) 
kshell = kshell_base * (kshell_norm / kshell_base[int_val]) 

# now these imaginary RIs are transformed and scaled to real RI:
nshell = 1.10 + np.imag(analytic_signal(kshell))
ncore = 1.02 + np.imag(analytic_signal(kcore))

# for the d'Milay routine:
mshell = nshell - kshell*1j
mcore = ncore - kcore*1j

# Gladstone Dale equivalents for a homogenous sphere:

# imaginary refractive index 
khom = kcore*Vc + kshell*Vs 
# real refractive index
nhom = ncore*Vc + nshell*Vs

### This section defines the particle size distribution as well as the angular variables:

In [18]:
psd = np.arange(1,101,1)
deltad=1

# Don't change any of this:
theta901 = np.arange(0, 90.1, 0.1) 
nang=901
angles=np.cos(np.deg2rad(theta901)) 
theta1 = np.deg2rad(theta901) 
theta2 = np.flipud(np.pi-np.deg2rad(theta901[0:900]))
theta=np.append(theta1,theta2)
back1=np.where(theta==(np.pi/2)) 
back2=np.where(theta==(np.pi))
d1=np.diff(theta)
dtheta = np.append(d1, d1[0])

### Declaring variables to be filled:

In [19]:
VSF = np.zeros((len(D_eff),len(l), 1801))   # dimensions jjj(deff), nii(wavelength), 1801 angles    
PF_check = np.zeros((len(D_eff),len(l)))
d_alpha = []  
PF = np.zeros((len(D_eff), len(wavelength), 1801))

# declare all lists and arrays to fill in the jjj loop (refilled each iteration)
Qc, Sigma_c, c, Qb, Sigma_b, b, Qa, Sigma_a, a, Qbb, Sigma_bb, bb, bbtilde = (np.zeros([len(D_eff),len(l)]) for i in range(13))
a_sol, a_solm, Qastar2_dir, Qas21 = (np.zeros([len(D_eff),len(l)]) for i in range(4))

 ### Then the loops. 
 
 They are structured as follows:
 
 nii - for each wavelength:
 
 > jj - for each size particle in the psd, do the diMilay, get the bbprob for each particle:
 
 > jjj - for each Deff requested, calc the resulting efficiency factors and phase fns: 
 >>  ai - for each of 1801 angles, for each Deff, calculate the VSF
 
 > then check the sum of phase functions (should be close to 1), b and bb for each wavelength, for each Deff
 
 end

In [24]:
for nii in np.arange(0,len(l)): # this is the wavelength loop
    print(nii)

    # declare lists to be filled on each iteration of the psd loop
    II, phaseMB, alpha, bbprob, bbprob1, Qbbro, checkMB, Qbro, Qcro, M1 = ([] for i in range (10))
    
    for jj in np.arange(0, len(psd)): # this is the psd loop

        [QEXT,QSCA,GQSC,QBS,m1,m2,s21,d21] = Dmmex_R14B_4.dmilay((psd[jj]*FR)/2,psd[jj]/2,wvno[nii],mshell[nii],mcore[nii],angles,901,901)
        # the diMilay calculates each particle's optical density, first step in getting the efficiency factors
        # on each iteration of jj, we get a different QEXT and QSCA out. So these must be stored in their own array
        Qcro.insert(jj,QEXT) 
        Qbro.insert(jj,QSCA)    
        
        m1_seta = [num[0] for num in m1]
        m1_setb = [num[1] for num in m1]
        M1 = np.append(m1_seta,m1_setb[900:0:-1])
        M2 = np.append(m2[0:901,0], m2[900:0:-1,1])
        myval = (M1+M2)/2 
        II.insert(jj, myval) 
        
        alpha2=2*np.pi*(psd[jj]/2)/wavelength[nii] 
        alpha.insert(jj, alpha2) 

        phaseMB_jj = [II[jj] / (np.pi* Qbro[jj]* (alpha[jj]**2))]
        phaseMB.insert(jj,phaseMB_jj)

        checkMB_jj = [2* np.pi* np.sum(phaseMB_jj * np.sin(theta) * dtheta)]
        checkMB.insert(jj,checkMB_jj)

        section_jj = [item[900:1801] for item in phaseMB_jj]
        bbprob_jj = 2*np.pi* np.sum((section_jj *np.sin(theta[900:1801]) *dtheta[900:1801]))
        bbprob.insert(jj, bbprob_jj) 
        Qbbro_jj = QSCA * bbprob_jj 
        Qbbro.insert(jj,Qbbro_jj) 
    
	# we are still in the nii loop here! just the jj loop has ended

    d_alpha_nii = alpha[1] - alpha[0]
    d_alpha.insert(nii,d_alpha_nii)

	# jjj loop starts here
    for jjj in np.arange(0,len(D_eff)):

        # for standard normal distribution 
        exponent = (-psd/ 2)/ ((D_eff[jjj]/ 2) * V_eff)
        psd2 = 1.0e20 * np.power((psd/2),((1-3* V_eff)/V_eff)) * np.exp(exponent)
        # The 1e20 above is for scaling to plot, it gets normalised afterwards.
        # Here (into the psd2 variable) you could also put in your own distribution
        
        psdm1 = psd / 1e6; # because it was in micron 1:1:100   
        civol = np.pi/ 6 * sum(psd2 * psdm1 **3 * deltad) # the total volume of the distribution - before normalising
        psdm2 = psd2 * (1./ (civol * ci)) # normalising to the ci, i.e. making the distribution chl-specific
        psdvol = np.pi/ 6 * sum(psdm2 * np.power(psdm1, 3) * deltad) # resulting volume of the distribution
        indiv_vols = (4/3 * np.pi * (psdm1/2) ** 3)* psdm2 # individual size bin volumes
        
        # calculating the optical efficiencies (proportion of incident light absorbed, scattered on particle cross section) 
        Qc[jjj, nii] = sum(Qcro *psdm2 * np.power(psdm1,2) * deltad)/ sum(psdm2 * np.power(psdm1,2) *deltad)
        Sigma_c[jjj,nii] = np.pi/4 * Qc[jjj, nii] * sum(np.power(psdm1, 2) * deltad)
        c[jjj,nii] = np.pi/4* Qc[jjj, nii]* sum(psdm2* np.power(psdm1,2)* deltad)
        
        Qb[jjj, nii] = sum(Qbro * psdm2 * np.power(psdm1,2) * deltad) /sum(psdm2* np.power(psdm1,2)* deltad) 	            
        Sigma_b[jjj,nii] = np.pi/4 * Qb[jjj,nii]* sum(np.power(psdm1,2)* deltad)
        b[jjj, nii] = np.pi/4* Qb[jjj, nii]* sum(psdm2* np.power(psdm1,2)* deltad)

        Qbb[jjj, nii] = sum(Qbbro * psdm2 * np.power(psdm1,2) * deltad) /sum(psdm2* np.power(psdm1,2)* deltad)
        Sigma_bb[jjj, nii] = np.pi/4 * Qbb[jjj, nii] * sum(np.power(psdm1, 2) * deltad)
        bb[jjj, nii] =  np.pi/4* Qbb[jjj, nii]* sum(psdm2 * np.power(psdm1, 2) * deltad)
        
        Qa[jjj, nii] = Qc[jjj, nii] - Qb[jjj, nii]
        Sigma_a[jjj, nii] = np.pi/4 * Qa[jjj, nii]* sum(np.power(psdm1,2)* deltad)
        a[jjj, nii] = c[jjj, nii] - b[jjj, nii]

        betabar, VSF_1 = ([] for i in range(2))
        checkbar = []
        b_check, bb_check = (np.zeros((len(D_eff),len(wavelength))) for i in range(2))
        
        # bb fraction/probability
        bbtilde[jjj, nii] = bb[jjj, nii] / b[jjj, nii]
			
		# this little sub loop is INSIDE the jjj loop		
        for ai in range (0, nang * 2 - 1):  
            varII = [item[ai] for item in II]
            betabar_ai = (1 / np.pi) * (sum(varII * psdm2 * d_alpha[nii]) / sum(Qbro * psdm2 * np.power(alpha, 2) * d_alpha[nii]))
            betabar.insert(ai, betabar_ai)
            VSF_1_ai = betabar[ai] * b[jjj, nii]  
            VSF_1.insert(ai, VSF_1_ai) 
            # this gives VSF_1 of 1801 angles. For the current instance of nii (wavelength) and jjj (Deff)

        # checkbar is back outside of the sub loop
        checkbar = (2* np.pi * sum(betabar * np.sin(theta) * dtheta))
        PF_check[jjj,nii] = checkbar
        
        b_check[jjj,nii] = 2 * np.pi * sum((VSF_1) * np.sin(theta) * dtheta)
        
        PF[jjj,nii,:] = betabar
        VSF[jjj,nii,:] = VSF_1 # VSF_1s are put into matrix on each iteration of Deff, then wavelength.
        
        # We want to get out all wavelengths but backscatter only angles for each Deff:    
        slice_obj = slice(900, 1801) 
        # want to get the backward angles for this instance of Deff and all the wavelengths:
        VSF_b = VSF[jjj, nii, slice_obj]
        
        bb_check[jjj,nii] = 2 * np.pi * sum((VSF_b) * np.sin(theta[900: 1801]) * dtheta[900: 1801])      
        
        # both the jjj loop and the nii loop end here.

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [11]:
# save output to mat file:

output_2lay = {}
output_2lay['a'] = a
output_2lay['b'] = b
output_2lay['bb'] = bb
output_2lay['D_eff'] = D_eff
output_2lay['ci'] = ci
output_2lay['pdsm2'] = psdm2
output_2lay['PF'] = PF
output_2lay['theta'] = theta
output_2lay['Sigma_a'] = Sigma_a
output_2lay['Sigma_bb'] = Sigma_bb
output_2lay['Sigma_b'] = Sigma_b

io.savemat('/.../output_2lay.mat', output_2lay) # insert your filepath